In [1]:
from google.colab import files
uploaded = files.upload()

Saving kor_var.db to kor_var.db


In [80]:
import pandas as pd
import sqlite3

#DB 연결
conn = sqlite3.connect('kor_var.db')
cur = conn.cursor()
var = pd.read_sql('SELECT * FROM kor_var', conn)
var.columns=['Id','날짜','코로나 일일확진자(명)','코로나전일대비(명)','환율(원)','환율전일대비(원)','유가,WTI(달러)','유가전일대비(달러)','미국,S&P 500','미국 등락률(%)']

var['날짜']=pd.to_datetime(var['날짜'])

var

,Id,날짜,코로나 일일확진자(명),코로나전일대비(명),환율(원),환율전일대비(원),"유가,WTI(달러)",유가전일대비(달러),"미국,S&P 500",미국 등락률(%)
0,0,2020-11-20,348.0,23.0,1117.0,NaN,42.15,NaN,3581.87,0.39
1,1,2020-11-21,386.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2,2020-11-22,330.0,-56.0,NaN,NaN,NaN,NaN,NaN,NaN
3,3,2020-11-23,271.0,-59.0,1112.0,-5.0,43.06,0.91,3557.54,-0.68
4,4,2020-11-24,349.0,78.0,1110.0,-2.0,44.91,1.85,3577.59,0.56
...,...,...,...,...,...,...,...,...,...,...
405,405,2020-10-26,NaN,NaN,NaN,NaN,NaN,NaN,3465.39,0.34
406,406,2020-10-23,NaN,NaN,NaN,NaN,NaN,NaN,3453.49,0.52
407,407,2020-10-22,NaN,NaN,NaN,NaN,NaN,NaN,3435.56,-0.22
408,408,2020-10-21,NaN,NaN,NaN,NaN,NaN,NaN,3443.12,0.47


In [4]:
pip install pykrx

     |████████████████████████████████| 87 kB 3.1 MB/s 
     |████████████████████████████████| 60 kB 5.3 MB/s 
     |████████████████████████████████| 251 kB 40.3 MB/s 


In [75]:
from pykrx import stock
#음식료품 주가
food = stock.get_index_ohlcv("20201120", "20550228", "1005")
food['등락률']=( food['종가']-food['종가'].shift())/food['종가'].shift() *100
food=food.reset_index(drop=False)
food

음식료품,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,2020-11-20,3861.31,3876.83,3853.62,3858.97,33123825,238845333793,NaN
1,2020-11-23,3872.22,3880.86,3845.62,3859.26,120918660,190274199050,0.007515
2,2020-11-24,3860.82,3866.18,3823.39,3832.16,208710593,230737458447,-0.702207
3,2020-11-25,3837.03,3847.01,3780.25,3802.82,43692451,179425671158,-0.765626
4,2020-11-26,3795.78,3819.33,3795.78,3810.20,737620851,417743997940,0.194067
...,...,...,...,...,...,...,...,...
257,2021-12-06,3707.99,3734.39,3666.47,3717.59,5910378,61180319291,0.102591
258,2021-12-07,3717.08,3783.62,3716.13,3780.37,5288251,72329307980,1.688728
259,2021-12-08,3796.91,3803.89,3739.92,3741.42,12862841,70890953722,-1.030322
260,2021-12-09,3755.27,3777.85,3748.89,3771.11,16361909,92398163626,0.793549


In [227]:
food_var = pd.merge(food,var, how='left',on='날짜')

food_var=food_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
food_var = food_var.dropna() #결측치 있는 행 제거
food_var=food_var.reset_index(drop=True)

food_var

,등락률,코로나 일일확진자(명),코로나전일대비(명),환율(원),환율전일대비(원),"유가,WTI(달러)",유가전일대비(달러),"미국,S&P 500",미국 등락률(%)
0,0.007515,271.0,-59.0,1112.0,-5.0,43.06,0.91,3557.54,-0.68
1,-0.702207,349.0,78.0,1110.0,-2.0,44.91,1.85,3577.59,0.56
2,-0.765626,382.0,33.0,1107.0,-3.0,45.71,0.80,3635.41,1.62
3,0.284762,555.0,-26.0,1105.0,-2.5,45.53,-0.18,3629.65,-0.16
4,-0.929849,438.0,-12.0,1107.5,2.5,45.34,-0.19,3638.35,0.24
...,...,...,...,...,...,...,...,...,...
248,0.102591,4324.0,-802.0,1182.0,-1.0,69.49,3.23,4538.43,-0.84
249,1.688728,4954.0,630.0,1179.0,-3.0,72.05,2.56,4591.67,1.17
250,-1.030322,7174.0,2220.0,1177.0,-2.0,72.36,0.31,4686.75,2.07
251,0.793549,7102.0,-72.0,1176.5,-0.5,70.94,-1.42,4701.21,0.31


In [243]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(food_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = food_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]

X_train.shape,y_train.shape, X_val.shape, y_val.shape

((202, 8), (202,), (51, 8), (51,))

In [229]:
pip install category_encoders

In [244]:
from sklearn.linear_model import RidgeCV
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

alphas = [0, 0.001, 0.01, 0.1] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

#alpha:  1.0
#best score:  0.0039915234669229745

#alpha:  0.1
#best score:  -0.009420379578308746

#alpha:  0.01
#best score:  -0.021511171057986434

alpha:  0.1
best score:  -0.009420379578308746


In [245]:
y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

#0.7922393715611292 0
# 0.789415006430353
#0.7984414916166488

#0.07131910552677889
#0.07784692394121029
#0.07513518588144852

0.789415006430353 0.07784692394121029


In [246]:
pip install joblib

In [247]:
import joblib
joblib.dump(ridge, 'model_food.pkl')

['model_food.pkl']

In [232]:
from sklearn.linear_model import LinearRegression
#선형회귀모델
model = LinearRegression()

#모델 fit
model.fit(X_train, y_train)
y_pred = model.predict(X_train)

#테스트 데이터에 적용
y_pred = model.predict(X_val)

In [226]:
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

0.800376369530565 0.07487460356352349


In [252]:
#1006 섬유의복
clo = stock.get_index_ohlcv("20201120", "20550228", "1006")
clo['등락률']=( clo['종가']-clo['종가'].shift())/clo['종가'].shift() *100
clo=clo.reset_index(drop=False)
clo

섬유의복,날짜,시가,고가,저가,종가,거래량,거래대금,등락률
0,2020-11-20,252.90,255.47,252.82,255.15,11735966,30668229260,NaN
1,2020-11-23,254.80,254.80,250.66,251.88,9255009,49911102252,-1.281599
2,2020-11-24,252.61,254.00,251.99,253.48,18004855,41274674819,0.635223
3,2020-11-25,254.31,255.16,249.86,251.88,17016877,48841480707,-0.631214
4,2020-11-26,250.78,251.70,248.11,248.66,8320461,31834505810,-1.278387
...,...,...,...,...,...,...,...,...
257,2021-12-06,501.28,501.28,487.93,494.74,48110967,99008639483,-1.893752
258,2021-12-07,493.67,500.57,492.56,498.61,33335124,101318914505,0.782229
259,2021-12-08,503.25,506.50,493.77,496.17,29904118,73469959513,-0.489360
260,2021-12-09,498.21,506.40,496.86,502.20,64496171,130022527965,1.215309


In [253]:
clo_var = pd.merge(clo,var, how='left',on='날짜')

clo_var=clo_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
clo_var = clo_var.dropna() #결측치 있는 행 제거
clo_var=clo_var.reset_index(drop=True)

clo_var

,등락률,코로나 일일확진자(명),코로나전일대비(명),환율(원),환율전일대비(원),"유가,WTI(달러)",유가전일대비(달러),"미국,S&P 500",미국 등락률(%)
0,-1.281599,271.0,-59.0,1112.0,-5.0,43.06,0.91,3557.54,-0.68
1,0.635223,349.0,78.0,1110.0,-2.0,44.91,1.85,3577.59,0.56
2,-0.631214,382.0,33.0,1107.0,-3.0,45.71,0.80,3635.41,1.62
3,-0.237272,555.0,-26.0,1105.0,-2.5,45.53,-0.18,3629.65,-0.16
4,0.278147,438.0,-12.0,1107.5,2.5,45.34,-0.19,3638.35,0.24
...,...,...,...,...,...,...,...,...,...
248,-1.893752,4324.0,-802.0,1182.0,-1.0,69.49,3.23,4538.43,-0.84
249,0.782229,4954.0,630.0,1179.0,-3.0,72.05,2.56,4591.67,1.17
250,-0.489360,7174.0,2220.0,1177.0,-2.0,72.36,0.31,4686.75,2.07
251,1.215309,7102.0,-72.0,1176.5,-0.5,70.94,-1.42,4701.21,0.31


In [254]:
from sklearn.model_selection import train_test_split

df_train, df_val = train_test_split(clo_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = food_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]

X_train.shape,y_train.shape, X_val.shape, y_val.shape

((202, 8), (202,), (51, 8), (51,))

In [256]:
from sklearn.linear_model import RidgeCV
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # FutureWarning 제거

alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

#alpha:  1.0
#best score:  0.0039915234669229745

#alpha:  0.1
#best score:  -0.009420379578308746

#alpha:  0.01
#best score:  -0.021511171057986434

alpha:  1.0
best score:  -0.07228700752904423


In [257]:
y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

#0.7922393715611292 0
# 0.789415006430353
#0.7984414916166488

#0.07131910552677889
#0.07784692394121029
#0.07513518588144852

1.1647026322089027 -0.018926095905775986


In [258]:
import joblib
joblib.dump(ridge, 'model_clo.pkl')

['model_clo.pkl']

In [259]:
from sklearn.linear_model import LinearRegression
#선형회귀모델
model = LinearRegression()

#모델 fit
model.fit(X_train, y_train)
y_pred = model.predict(X_train)

#테스트 데이터에 적용
y_pred = model.predict(X_val)

In [260]:
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

1.1841237084020397 -0.06236697538053204


In [261]:
#1007 종이목재
wood = stock.get_index_ohlcv("20201120", "20550228", "1007")
wood['등락률']=( wood['종가']-wood['종가'].shift())/wood['종가'].shift() *100
wood=wood.reset_index(drop=False)

wood_var = pd.merge(wood,var, how='left',on='날짜')

wood_var=wood_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
wood_var = wood_var.dropna() #결측치 있는 행 제거
wood_var=wood_var.reset_index(drop=True)

df_train, df_val = train_test_split(wood_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = wood_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_wood.pkl')

alpha:  1.0
best score:  -0.04498402258997598
1.3361695719591993 -0.020472055144902912


['model_wood.pkl']

In [262]:
#1008 화학
che = stock.get_index_ohlcv("20201120", "20550228", "1008")
che['등락률']=( che['종가']-che['종가'].shift())/che['종가'].shift() *100
che=che.reset_index(drop=False)

che_var = pd.merge(che,var, how='left',on='날짜')

che_var=che_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
che_var = che_var.dropna() #결측치 있는 행 제거
che_var=che_var.reset_index(drop=True)

df_train, df_val = train_test_split(che_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = che_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_che.pkl')

alpha:  1.0
best score:  0.07500232704801843
0.9526667967856067 0.02732302529418751


['model_che.pkl']

In [263]:
#1009 의약품, med
med = stock.get_index_ohlcv("20201120", "20550228", "1009")
med['등락률']=( med['종가']-med['종가'].shift())/med['종가'].shift() *100
med=med.reset_index(drop=False)

med_var = pd.merge(med,var, how='left',on='날짜')

med_var=med_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
med_var = med_var.dropna() #결측치 있는 행 제거
med_var=med_var.reset_index(drop=True)

df_train, df_val = train_test_split(med_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = med_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_med.pkl')

alpha:  1.0
best score:  -0.07158833893217678
1.397701949189045 -0.05490947722654105


['model_med.pkl']

In [264]:
#1010 비금속광물, nonmetal
nonmetal = stock.get_index_ohlcv("20201120", "20550228", "1010")
nonmetal['등락률']=( nonmetal['종가']-nonmetal['종가'].shift())/nonmetal['종가'].shift() *100
nonmetal=nonmetal.reset_index(drop=False)

nonmetal_var = pd.merge(nonmetal,var, how='left',on='날짜')

nonmetal_var=nonmetal_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
nonmetal_var = nonmetal_var.dropna() #결측치 있는 행 제거
nonmetal_var=nonmetal_var.reset_index(drop=True)

df_train, df_val = train_test_split(nonmetal_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = nonmetal_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_nonmetal.pkl')

alpha:  1.0
best score:  0.04924054688148751
1.0735936386869687 -0.07207384517786752


['model_nonmetal.pkl']

In [265]:
#1011 철강금속, ste
ste = stock.get_index_ohlcv("20201120", "20550228", "1011")
ste['등락률']=( ste['종가']-ste['종가'].shift())/ste['종가'].shift() *100
ste=ste.reset_index(drop=False)

ste_var = pd.merge(ste,var, how='left',on='날짜')

ste_var=ste_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
ste_var = ste_var.dropna() #결측치 있는 행 제거
ste_var=ste_var.reset_index(drop=True)

df_train, df_val = train_test_split(ste_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = ste_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_ste.pkl')

alpha:  1.0
best score:  0.05784675816995159
1.3302449839068076 -0.007816750469993305


['model_ste.pkl']

In [266]:
#1012 기계, mac
mac = stock.get_index_ohlcv("20201120", "20550228", "1012")
mac['등락률']=( mac['종가']-mac['종가'].shift())/mac['종가'].shift() *100
mac=mac.reset_index(drop=False)

mac_var = pd.merge(mac,var, how='left',on='날짜')

mac_var=mac_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
mac_var = mac_var.dropna() #결측치 있는 행 제거
mac_var=mac_var.reset_index(drop=True)

df_train, df_val = train_test_split(mac_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = mac_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_mac.pkl')

alpha:  1.0
best score:  0.03851162005232771
1.4410223626873004 0.0700827516323258


['model_mac.pkl']

In [267]:
#1013 전기전자, ele
ele = stock.get_index_ohlcv("20201120", "20550228", "1013")
ele['등락률']=( ele['종가']-ele['종가'].shift())/ele['종가'].shift() *100
ele=ele.reset_index(drop=False)

ele_var = pd.merge(ele,var, how='left',on='날짜')

ele_var=ele_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
ele_var = ele_var.dropna() #결측치 있는 행 제거
ele_var=ele_var.reset_index(drop=True)

df_train, df_val = train_test_split(ele_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = ele_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_ele.pkl')

alpha:  0.1
best score:  0.1898026412932413
0.9597003591429368 0.08561532671397887


['model_ele.pkl']

In [269]:
#1014 의료정밀, prec
prec = stock.get_index_ohlcv("20201120", "20550228", "1014")
prec['등락률']=( prec['종가']-prec['종가'].shift())/prec['종가'].shift() *100
prec=prec.reset_index(drop=False)

prec_var = pd.merge(prec,var, how='left',on='날짜')

prec_var=prec_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
prec_var = prec_var.dropna() #결측치 있는 행 제거
prec_var=prec_var.reset_index(drop=True)

df_train, df_val = train_test_split(prec_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = prec_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_prec.pkl')

alpha:  1.0
best score:  0.024721351474821397
1.2390170195485084 -0.12746430462425695


['model_prec.pkl']

In [270]:
#1015 운수장비, trans
trans = stock.get_index_ohlcv("20201120", "20550228", "1015")
trans['등락률']=( trans['종가']-trans['종가'].shift())/trans['종가'].shift() *100
trans=trans.reset_index(drop=False)

trans_var = pd.merge(trans,var, how='left',on='날짜')

trans_var=trans_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
trans_var = trans_var.dropna() #결측치 있는 행 제거
trans_var=trans_var.reset_index(drop=True)

df_train, df_val = train_test_split(trans_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = trans_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_trans.pkl')

alpha:  1.0
best score:  0.034746422435343274
1.3254891135458278 0.030288176878095263


['model_trans.pkl']

In [271]:
#1016 유통업, cir
cir = stock.get_index_ohlcv("20201120", "20550228", "1016")
cir['등락률']=( cir['종가']-cir['종가'].shift())/cir['종가'].shift() *100
cir=cir.reset_index(drop=False)

cir_var = pd.merge(cir,var, how='left',on='날짜')

cir_var=cir_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
cir_var = cir_var.dropna() #결측치 있는 행 제거
cir_var=cir_var.reset_index(drop=True)

df_train, df_val = train_test_split(cir_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = cir_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_cir.pkl')

alpha:  1.0
best score:  0.11460043802968851
0.896283676366834 0.09725218722064144


['model_cir.pkl']

In [272]:
#1017 전기가스업, gas
gas = stock.get_index_ohlcv("20201120", "20550228", "1017")
gas['등락률']=( gas['종가']-gas['종가'].shift())/gas['종가'].shift() *100
gas=gas.reset_index(drop=False)

gas_var = pd.merge(gas,var, how='left',on='날짜')

gas_var=gas_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
gas_var = gas_var.dropna() #결측치 있는 행 제거
gas_var=gas_var.reset_index(drop=True)

df_train, df_val = train_test_split(gas_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = gas_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_gas.pkl')

alpha:  1.0
best score:  0.0021987142936504833
1.1789063425811845 0.004558613706801995


['model_gas.pkl']

In [273]:
#1018 건설업, con
con = stock.get_index_ohlcv("20201120", "20550228", "1018")
con['등락률']=( con['종가']-con['종가'].shift())/con['종가'].shift() *100
con=con.reset_index(drop=False)

con_var = pd.merge(con,var, how='left',on='날짜')

con_var=con_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
con_var = con_var.dropna() #결측치 있는 행 제거
con_var=con_var.reset_index(drop=True)

df_train, df_val = train_test_split(con_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = con_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_con.pkl')

alpha:  0.1
best score:  0.10023397183266144
1.1271257978893885 -0.0033795808389571746


['model_con.pkl']

In [274]:
#1019 운수창고업, ware
ware = stock.get_index_ohlcv("20201120", "20550228", "1019")
ware['등락률']=( ware['종가']-ware['종가'].shift())/ware['종가'].shift() *100
ware=ware.reset_index(drop=False)

ware_var = pd.merge(ware,var, how='left',on='날짜')

ware_var=ware_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
ware_var = ware_var.dropna() #결측치 있는 행 제거
ware_var=ware_var.reset_index(drop=True)

df_train, df_val = train_test_split(ware_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = ware_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_ware.pkl')

alpha:  1.0
best score:  -0.004134174895069243
1.2585167878103638 0.016390992394159865


['model_ware.pkl']

In [275]:
#1020 통신업, tel
tel = stock.get_index_ohlcv("20201120", "20550228", "1020")
tel['등락률']=( tel['종가']-tel['종가'].shift())/tel['종가'].shift() *100
tel=tel.reset_index(drop=False)

tel_var = pd.merge(tel,var, how='left',on='날짜')

tel_var=tel_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
tel_var = tel_var.dropna() #결측치 있는 행 제거
tel_var=tel_var.reset_index(drop=True)

df_train, df_val = train_test_split(tel_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = tel_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_tel.pkl')

alpha:  1.0
best score:  -0.04528353149609425
1.244478278591293 0.007123983882411977


['model_tel.pkl']

In [276]:
#1021 금융업, fin
fin = stock.get_index_ohlcv("20201120", "20550228", "1021")
fin['등락률']=( fin['종가']-fin['종가'].shift())/fin['종가'].shift() *100
fin=fin.reset_index(drop=False)

fin_var = pd.merge(fin,var, how='left',on='날짜')

fin_var=fin_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
fin_var = fin_var.dropna() #결측치 있는 행 제거
fin_var=fin_var.reset_index(drop=True)

df_train, df_val = train_test_split(fin_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = fin_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_fin.pkl')

alpha:  1.0
best score:  0.07423583005855328
0.8648052357661976 -0.012249556980881815


['model_fin.pkl']

In [277]:
#1022 은행, bank
bank = stock.get_index_ohlcv("20201120", "20550228", "1022")
bank['등락률']=( bank['종가']-bank['종가'].shift())/bank['종가'].shift() *100
bank=bank.reset_index(drop=False)

bank_var = pd.merge(bank,var, how='left',on='날짜')

bank_var=bank_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
bank_var = bank_var.dropna() #결측치 있는 행 제거
bank_var=bank_var.reset_index(drop=True)

df_train, df_val = train_test_split(bank_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = bank_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_bank.pkl')

alpha:  1.0
best score:  -0.014001009414975684
1.2591934027452287 -0.18895738727517775


['model_bank.pkl']

In [278]:
#1024 증권, sto
sto = stock.get_index_ohlcv("20201120", "20550228", "1024")
sto['등락률']=( sto['종가']-sto['종가'].shift())/sto['종가'].shift() *100
sto=sto.reset_index(drop=False)

sto_var = pd.merge(sto,var, how='left',on='날짜')

sto_var=sto_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
sto_var = sto_var.dropna() #결측치 있는 행 제거
sto_var=sto_var.reset_index(drop=True)

df_train, df_val = train_test_split(sto_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = sto_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_sto.pkl')

alpha:  0.1
best score:  0.13087085657501077
1.0566143074715275 0.05550743850334694


['model_sto.pkl']

In [279]:
#1025 보험, ins
ins = stock.get_index_ohlcv("20201120", "20550228", "1025")
ins['등락률']=( ins['종가']-ins['종가'].shift())/ins['종가'].shift() *100
ins=ins.reset_index(drop=False)

ins_var = pd.merge(ins,var, how='left',on='날짜')

ins_var=ins_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
ins_var = ins_var.dropna() #결측치 있는 행 제거
ins_var=ins_var.reset_index(drop=True)

df_train, df_val = train_test_split(ins_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = ins_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_ins.pkl')

alpha:  1.0
best score:  -0.02049727460463511
1.019351107117209 -0.005094670523397182


['model_ins.pkl']

In [280]:
#1026 서비스업, ser
ser = stock.get_index_ohlcv("20201120", "20550228", "1026")
ser['등락률']=( ser['종가']-ser['종가'].shift())/ser['종가'].shift() *100
ser=ser.reset_index(drop=False)

ser_var = pd.merge(ser,var, how='left',on='날짜')

ser_var=ser_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
ser_var = ser_var.dropna() #결측치 있는 행 제거
ser_var=ser_var.reset_index(drop=True)

df_train, df_val = train_test_split(ser_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = ser_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_ser.pkl')

alpha:  1.0
best score:  -0.026991085971601315
1.0725091372910336 0.08182137057707539


['model_ser.pkl']

In [281]:
#1027 제조업, man
man = stock.get_index_ohlcv("20201120", "20550228", "1027")
man['등락률']=( man['종가']-man['종가'].shift())/man['종가'].shift() *100
man=man.reset_index(drop=False)

man_var = pd.merge(man,var, how='left',on='날짜')

man_var=man_var[['등락률','코로나 일일확진자(명)',	'코로나전일대비(명)',	'환율(원)',	'환율전일대비(원)',	'유가,WTI(달러)',	'유가전일대비(달러)',	'미국,S&P 500',	'미국 등락률(%)']]
man_var = man_var.dropna() #결측치 있는 행 제거
man_var=man_var.reset_index(drop=True)

df_train, df_val = train_test_split(man_var, test_size= 0.2, random_state = 2)

target ='등락률'
features = man_var.drop(columns = [target]).columns

X_train = df_train[features]
y_train = df_train[target]
X_val = df_val[features]
y_val = df_val[target]


alphas = [0, 0.001, 0.01, 0.1, 1.0] #알파 = 람다(페널티). 

ridge = RidgeCV(alphas=alphas, normalize=True, cv=5) #cv: 교차검증(Cross-validation)
ridge.fit(X_train, y_train)
print("alpha: ", ridge.alpha_)
print("best score: ", ridge.best_score_)

y_pred = ridge.predict(X_val)

#테스트 타깃 데이터와 예측데이터를 가지고 평균 절대 오차 구하기
from sklearn.metrics import mean_absolute_error, r2_score
mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(mae, r2)

import joblib
joblib.dump(ridge, 'model_man.pkl')

alpha:  0.1
best score:  0.2201123515893464
0.8079008134023399 0.06704219143546253


['model_man.pkl']